<a href="https://colab.research.google.com/github/thomasmcz/evapotranspiration/blob/main/STARFM%2BSEBAL_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INITIALIZING EE

In [ ]:
# Import Earth Engine Library
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

# Importing libraries and defining functions

In [ ]:
# Import some essencial libraries and define the functions for focal analyses (make_slices) and image export (make_raster)
 
import numpy as np
from osgeo import gdal
import os
import math
 
os.chdir(r"/content/drive/MyDrive")

# The next two functions were obtained from "Geoprocessing with Python" book, by Chris Garrard
# It is available from https://www.manning.com/books/geoprocessing-with-python?query=geoprocessing
def make_slices(data, win_size):
    """Return a list of slices given a window size.
    data     - two-dimensional array to get slices from
    win_size - tuple of (rows, columns) for the moving window"""
    rows = data.shape[0] - win_size[0] + 1
    cols = data.shape[1] - win_size[1] + 1
    slices = []
    for i in range(win_size[0]):
        for j in range(win_size[1]):
            slices.append(data[i:rows+i, j:cols+j])
    return slices
 
def make_raster(in_ds, fn, data, data_type, nodata=None):
    """Create a one-band GeoTIFF.
    in_ds - datasource to copy projection and geotransform from
    fn - path to the file to create
    data - NumPy array containing data to write
    data_type - output data type
    nodata - optional NoData value"""
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(fn, in_ds.RasterXSize, in_ds.RasterYSize, 1, data_type)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())
    out_band = out_ds.GetRasterBand(1)
    if nodata is not None:
        out_band.SetNoDataValue(nodata)
    out_band.WriteArray(data)
    out_band.FlushCache()
    out_band.ComputeStatistics(False)
    return out_ds

# Exporting MODIS images to your drive

In [ ]:
# Define your study area
mandacaru = ee.Geometry.Polygon(
        [[[-40.443765656859135, -9.369681936878271],
          [-40.443765656859135, -9.4184573955032],
          [-40.38179589977906, -9.4184573955032],
          [-40.38179589977906, -9.369681936878271]]])

#Choose the dates of the MODIS images that will be processed in the STARFM algorithm
data1 = ee.String('2016_10_29')
data2 = ee.String('2016_12_31')
data3 = ee.String('2017_01_17')

nome_gq = ee.String('MODIS/006/MYD09GQ/')
nome_a1 = ee.String('MODIS/006/MYD11A1/')
nome_a3 = ee.String('MODIS/006/MCD43A3/')

pegar_gq_1 = nome_gq.cat(data1)
pegar_gq_2 = nome_gq.cat(data2)
pegar_gq_3 = nome_gq.cat(data3)

pegar_a1_1 = nome_a1.cat(data1)
pegar_a1_2 = nome_a1.cat(data2)
pegar_a1_3 = nome_a1.cat(data3)

pegar_a3_1 = nome_a3.cat(data1)
pegar_a3_2 = nome_a3.cat(data2)
pegar_a3_3 = nome_a3.cat(data3)

img1 = ee.Image(pegar_gq_1.getInfo())
img2 = ee.Image(pegar_gq_2.getInfo())
img3 = ee.Image(pegar_gq_3.getInfo())

img4 = ee.Image(pegar_a1_1.getInfo())
img5 = ee.Image(pegar_a1_2.getInfo())
img6 = ee.Image(pegar_a1_3.getInfo())

img7 = ee.Image(pegar_a3_1.getInfo())
img8 = ee.Image(pegar_a3_2.getInfo())
img9 = ee.Image(pegar_a3_3.getInfo())

img_export1 = img1.select(['sur_refl_b01', 'sur_refl_b02']).multiply(0.0001)
img_export2 = img2.select(['sur_refl_b01', 'sur_refl_b02']).multiply(0.0001)
img_export3 = img3.select(['sur_refl_b01', 'sur_refl_b02']).multiply(0.0001)

img_export4 = img4.select(['LST_Day_1km']).multiply(0.02)
img_export5 = img5.select(['LST_Day_1km']).multiply(0.02)
img_export6 = img6.select(['LST_Day_1km']).multiply(0.02)

img_export7 = img7.select(['Albedo_BSA_shortwave']).multiply(0.001)
img_export8 = img8.select(['Albedo_BSA_shortwave']).multiply(0.001)
img_export9 = img9.select(['Albedo_BSA_shortwave']).multiply(0.001)

produto_1 = ee.String('modis_gq')
produto_2 = ee.String('modis_a1')
produto_3 = ee.String('modis_a3')

saida_1 = produto_1.cat(data1)
saida_2 = produto_1.cat(data2)
saida_3 = produto_1.cat(data3)

saida_4 = produto_2.cat(data1)
saida_5 = produto_2.cat(data2)
saida_6 = produto_2.cat(data3)

saida_7 = produto_3.cat(data1)
saida_8 = produto_3.cat(data2)
saida_9 = produto_3.cat(data3)

task1 = ee.batch.Export.image.toDrive(image=img_export1, folder='colab_imagens', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_1.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task2 = ee.batch.Export.image.toDrive(image=img_export2, folder='colab_imagens', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_2.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task3 = ee.batch.Export.image.toDrive(image=img_export3, folder='colab_imagens', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_3.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task4 = ee.batch.Export.image.toDrive(image=img_export4, folder='colab_imagens', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_4.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task5 = ee.batch.Export.image.toDrive(image=img_export5, folder='colab_imagens', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_5.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task6 = ee.batch.Export.image.toDrive(image=img_export6, folder='colab_imagens', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_6.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task7 = ee.batch.Export.image.toDrive(image=img_export7, folder='colab_imagens', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_7.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task8 = ee.batch.Export.image.toDrive(image=img_export8, folder='colab_imagens', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_8.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task9 = ee.batch.Export.image.toDrive(image=img_export9, folder='colab_imagens', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_9.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')

task1.start()
task1.status()

task2.start()
task2.status()

task3.start()
task3.status()

task4.start()
task4.status()

task5.start()
task5.status()

task6.start()
task6.status()

task7.start()
task7.status()

task8.start()
task8.status()

task9.start()
task9.status()

print('End of the Script')


Fim do Script


# Exporting Landsat-8 images to your drive

In [ ]:
# Define your study area
mandacaru = ee.Geometry.Polygon(
        [[[-40.443765656859135, -9.369681936878271],
          [-40.443765656859135, -9.4184573955032],
          [-40.38179589977906, -9.4184573955032],
          [-40.38179589977906, -9.369681936878271]]])

#Choose the dates of the Landsat images that will be processed in the STARFM algorithm
data1 = ee.String('2016_10_29')
data2 = ee.String('2017_01_17')

# Change the dates in the .filterDate method by adding the day that you want, following by the next day
img = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2016-10-29', '2016-10-30').filterBounds(mandacaru)
img = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2016-10-29', '2016-10-30').filterBounds(mandacaru)
img2 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2017-01-17', '2017-01-18').filterBounds(mandacaru)
raw = ee.ImageCollection('LANDSAT/LC08/C01/T1').filterDate('2016-10-29', '2016-10-30').filterBounds(mandacaru)
raw2 = ee.ImageCollection('LANDSAT/LC08/C01/T1').filterDate('2017-01-17', '2017-01-18').filterBounds(mandacaru)

mosaico = img.mosaic()
mosaico2 = img2.mosaic()

surface_reflectance = mosaico.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']).multiply(0.0001)
surface_reflectance2 = mosaico2.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']).multiply(0.0001)
albedo = surface_reflectance.expression(
    '(0.2453*B2) + (0.0508*B3) + (0.1804*B4) + (0.3081*B5) + (0.1332*B6) + (0.0521*B7) + 0.0011',{'B2': surface_reflectance.select('B2'),'B3': surface_reflectance.select('B3'),'B4': surface_reflectance.select('B4'),'B5': surface_reflectance.select('B5'),'B6': surface_reflectance.select('B6'),'B7': surface_reflectance.select('B7'),})
albedo2 = surface_reflectance2.expression(
    '(0.2453*B2) + (0.0508*B3) + (0.1804*B4) + (0.3081*B5) + (0.1332*B6) + (0.0521*B7) + 0.0011',{'B2': surface_reflectance2.select('B2'),'B3': surface_reflectance2.select('B3'),'B4': surface_reflectance2.select('B4'),'B5': surface_reflectance2.select('B5'),'B6': surface_reflectance2.select('B6'),'B7': surface_reflectance2.select('B7'),})

add = ee.Number(raw.aggregate_mean('RADIANCE_ADD_BAND_10'))
mult = ee.Number(raw.aggregate_mean('RADIANCE_MULT_BAND_10'))

add2 = ee.Number(raw2.aggregate_mean('RADIANCE_ADD_BAND_10'))
mult2 = ee.Number(raw2.aggregate_mean('RADIANCE_MULT_BAND_10'))

mosaico_raw = raw.mosaic()
mosaico_raw2 = raw2.mosaic()

rad = mosaico_raw.expression('add + mult * nd',{'add': add, 'mult': mult, 'nd': mosaico_raw.select('B10')})
rad2 = mosaico_raw2.expression('add + mult * nd',{'add': add2, 'mult': mult2, 'nd': mosaico_raw2.select('B10')})

elevacao = raw.aggregate_mean('SUN_ELEVATION')
elevacao2 = raw2.aggregate_mean('SUN_ELEVATION')

#print('The average of the elevations is: ', elevacao)

cos_z = ee.Number(elevacao).multiply(math.pi).divide(180).sin()
cos_z2 = ee.Number(elevacao2).multiply(math.pi).divide(180).sin()

#print ('The sin(E) is: ', cos_z)

dt_s = raw.aggregate_mean('EARTH_SUN_DISTANCE')
dt_s2 = raw2.aggregate_mean('EARTH_SUN_DISTANCE')

#print('The Earth-Sun Distance is: ', dt_s)
dr = ee.Number(1).divide((ee.Number(dt_s)).pow(2))
dr2 = ee.Number(1).divide((ee.Number(dt_s2)).pow(2))

#print ('dr is: ', dr)

reflectance_toa = mosaico_raw.expression('(-0.10000000149011612 + 0.000019999999494757503 * nd) / (cos_z * dr)',{'nd': mosaico_raw,'cos_z': cos_z, 'dr': dr})
reflectance_toa2 = mosaico_raw2.expression('(-0.10000000149011612 + 0.000019999999494757503 * nd) / (cos_z * dr)',{'nd': mosaico_raw2,'cos_z': cos_z2, 'dr': dr2})

savi = reflectance_toa.expression('(1.1 * (NIR - RED)) / (0.1 + NIR + RED)',{'NIR': reflectance_toa.select('B5'), 'RED': reflectance_toa.select('B4'),})

savi2 = reflectance_toa2.expression('(1.1 * (NIR - RED)) / (0.1 + NIR + RED)',{'NIR': reflectance_toa2.select('B5'), 'RED': reflectance_toa2.select('B4'),})

iaf = savi.where(savi.lt(0.688000), savi.expression('(log((0.69 - savi) / 0.59)) * (-1) / 0.91',{'savi': savi.select('constant'),}))
iaf2 = iaf.where(savi.gt(0.688000), 6)
iafcorrigido = iaf2.where(savi.lt(0.000001), 0)

iaf_2 = savi2.where(savi2.lt(0.688000), savi2.expression('(log((0.69 - savi) / 0.59)) * (-1) / 0.91',{'savi': savi2.select('constant'),}))
iaf2_2 = iaf_2.where(savi2.gt(0.688000), 6)
iafcorrigido_2 = iaf2_2.where(savi2.lt(0.000001), 0)

e_nb = iafcorrigido.expression('0.97 + (0.0033 * iaf)', {'iaf': iafcorrigido.select('constant')})
e_nb2 = e_nb.where(iafcorrigido.gt(2.999999), 0.98)
e_nbcorrigido = e_nb2.where(iafcorrigido.lt(0.000001), 0.99)

e_nb_2 = iafcorrigido_2.expression('0.97 + (0.0033 * iaf)', {'iaf': iafcorrigido_2.select('constant')})
e_nb2_2 = e_nb_2.where(iafcorrigido_2.gt(2.999999), 0.98)
e_nbcorrigido2 = e_nb2_2.where(iafcorrigido_2.lt(0.000001), 0.99)

tsup = rad.expression('1321.08 / log(774.89 * e_nb / (radiance - 0.29) + 1)', {'e_nb': e_nbcorrigido.select('constant'), 'radiance': rad})
tsup2 = rad.expression('1321.08 / log(774.89 * e_nb / (radiance - 0.29) + 1)', {'e_nb': e_nbcorrigido2.select('constant'), 'radiance': rad2})

imagem_1 = ee.String('reflectance_')
imagem_2 = ee.String('albedo_')
imagem_3 = ee.String('tsup_')

saida_1 = imagem_1.cat(data1)
saida_2 = imagem_1.cat(data2)
saida_3 = imagem_2.cat(data1)
saida_4 = imagem_2.cat(data2)
saida_5 = imagem_3.cat(data1)
saida_6 = imagem_3.cat(data2)

task1 = ee.batch.Export.image.toDrive(image=surface_reflectance, folder='colab_imagens', description='Landsat', scale=30, region=mandacaru, fileNamePrefix=saida_1.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task2 = ee.batch.Export.image.toDrive(image=surface_reflectance2, folder='colab_imagens', description='Landsat', scale=30, region=mandacaru, fileNamePrefix=saida_2.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')

task3 = ee.batch.Export.image.toDrive(image=albedo, folder='colab_imagens', description='Landsat', scale=30, region=mandacaru, fileNamePrefix=saida_3.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task4 = ee.batch.Export.image.toDrive(image=albedo2, folder='colab_imagens', description='Landsat', scale=30, region=mandacaru, fileNamePrefix=saida_4.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')

task5 = ee.batch.Export.image.toDrive(image=tsup, folder='colab_imagens', description='Landsat', scale=30, region=mandacaru, fileNamePrefix=saida_5.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task6 = ee.batch.Export.image.toDrive(image=tsup2, folder='colab_imagens', description='Landsat', scale=30, region=mandacaru, fileNamePrefix=saida_6.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')

task1.start()
task2.start()
task3.start()
task4.start()
task5.start()
task6.start()
print('End of the Script')

# STARFM PART I

In [ ]:
# This script selects the Spectrally Similar Neighbor Pixels and exports them with Euclidean distance information.

# Define the directory that has the images to be processed.
os.chdir(r"/content/drive/MyDrive/colab_imagens")

in_ds = gdal.Open('reflectance_2016_10_29.tif')# Load the first LANDSAT image (the day before the predicted date)
in_band_lv = in_ds.GetRasterBand(3) # Get the red band
in_band_liv = in_ds.GetRasterBand(4)  # Get the IR band

in_ds_mv = gdal.Open('modis_gq2016_10_29.tif') # Load the first MODIS image (the day before the predicted date)
in_band_mv = in_ds_mv.GetRasterBand(1) # Get the red band
in_band_miv = in_ds_mv.GetRasterBand(2) # Get the IR band

in_ds_k2 = gdal.Open('reflectance_2017_01_17.tif')# Load the second LANDSAT image (the day after the predicted date)
in_band_lv_k2 = in_ds_k2.GetRasterBand(3) # Get the red band
in_band_liv_k2 = in_ds_k2.GetRasterBand(4) # Get the red band

in_ds_mv_k2 = gdal.Open('modis_gq2017_01_17.tif') # Load the second MODIS image (the day after the predicted date)
in_band_mv_k2 = in_ds_mv_k2.GetRasterBand(1) # Get the red band
in_band_miv_k2 = in_ds_mv_k2.GetRasterBand(2) # Get the IR band

in_ds_mv_ko = gdal.Open('modis_gq2016_12_31.tif') # Load the third MODIS image (predicted date)
in_band_mv_ko = in_ds_mv_ko.GetRasterBand(1) # Get the red band
in_band_miv_ko = in_ds_mv_ko.GetRasterBand(2) # Get the IR band

xsize = in_band_lv.XSize #número de colunas
ysize = in_band_lv.YSize #número de linhas

img_de_saida = []
for z in range(1, ysize+1):
  img_de_saida.append('n{}.tif'.format(z))

mw = 49 #moving window
lp = mw - 1 #linhas perdidas
ns = mw * mw #número de slices
lsp = int(lp / 2) #linhas superiores perdidas --------------------------------------------ALTERAR LINHAS 184 E 190
sc = ns // 2 #slice central
n = 6 #número de linhas a serem resolvidas (ANTERIORMENTE FOI UTILIZADO 6!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

for i in range(0, ysize, n): #início do loop p/ resolver usando chunks
    if n + lp + i < ysize: #o n° de linhas a ser lido (n + 48) + o nº de linhas lido anteriormente deve ser menor que o nº total de linhas
        rows = n + lp #p/ uam moving window de 49x49, perde-se 48 linhas, logo, p/ resolver 2 linhas, deve-se ler 50
    else:
        rows = ysize - i #se não restarem 50 linhas para serem lidas, significa que i=1180, logo restam apenas 49 linhas, que serão usadas para resolver a última linha
        if rows < mw: #se o número de linhas a ser lido for menor que a moving window (49) o script deve parar
            break

    in_data_lv = in_band_lv.ReadAsArray(0, i, xsize, rows).astype(np.float32) #lendo a partir da 1ª coluna e da i-linha todas as col e rows-linhas
    in_data_mv = in_band_mv.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_lv_k2 = in_band_lv_k2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_mv_k2 = in_band_mv_k2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_mv_ko = in_band_mv_ko.ReadAsArray(0, i, xsize, rows).astype(np.float32)

    in_data_liv = in_band_liv.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_miv = in_band_miv.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_liv_k2 = in_band_liv_k2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_miv_k2 = in_band_miv_k2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_miv_ko = in_band_miv_ko.ReadAsArray(0, i, xsize, rows).astype(np.float32)

    driver = gdal.GetDriverByName('GTiff') #"pegando" a extensão .tiff (usar 'HFA' para .img) para designá-la na imagem de saída    

    for nome in img_de_saida:
        out_ds_lv = driver.Create(nome, xsize, n, ns, gdal.GDT_Float32) #criando a imagem de saída, com todas as colunas, 2 linhas, 2401 camadas e no formato float32
        out_ds_lv.SetProjection(in_ds.GetProjection()) #atribuindo a mesma projeção da imagem de entrada, EPSG 32724

        in_gt = in_ds.GetGeoTransform() #pegando o GeoTransform (origem das coordenadas e tamanho do pixel) da imagem de entrada
        subset_ulx, subset_uly = gdal.ApplyGeoTransform(in_gt, 0, i+lsp) #atribuindo a duas variáveis o canto sup esq em que se encontra a linha central da moving window, p/ i=0 a linha central tem canto com x=0 e y=24
        out_gt = list(in_gt) #como o GetGeoTransform retorna uma tupla, cria-se uma lista com os mesmos parâmetros
        out_gt[0] = subset_ulx #substituindo o parâmetro da coluna
        out_gt[3] = subset_uly #substituindo o parâmetro da linha
        out_ds_lv.SetGeoTransform(out_gt) #aplicando o novo GeoTransform para a imagem de saída

        slices_lv = make_slices(in_data_lv, (mw, mw)) #criando os 2401 slices para a banda do vermelho do Lansat e do MODIS
        slices_mv = make_slices(in_data_mv, (mw, mw))
        slices_lv_k2 = make_slices(in_data_lv_k2, (mw, mw))
        slices_mv_k2 = make_slices(in_data_mv_k2, (mw, mw))
        slices_mv_ko = make_slices(in_data_mv_ko, (mw, mw))

        difabsv = [abs(valor - slices_lv[sc]) for valor in slices_lv] #usando list comprehension para subtrair de cada slice o slice central
        desvpadv = np.std(np.ma.dstack(slices_lv), 2) #tirando o desvio padrão na dimensão z, ou seja, o desvio de todos os pixels da moving window
        menorqdesvv = np.ma.masked_where(difabsv > desvpadv, slices_lv) #mascarando os pixels em que a diferença abs é maior que o desvio padrão

        difabsv_k2 = [abs(valor - slices_lv_k2[sc]) for valor in slices_lv_k2]
        desvpadv_k2 = np.std(np.ma.dstack(slices_lv_k2), 2)
        menorqdesvv_k2 = np.ma.masked_where(difabsv_k2 > desvpadv_k2, slices_lv_k2)

        filtro_lv_k1 = np.ma.masked_where(menorqdesvv + menorqdesvv_k2 == -1, menorqdesvv) #nenhuma soma será =-1, logo, o valor de menorqdesvv será preservado, mas apenas para os pixels que são neighbours para ambas as variáveis
        filtro_lv_k2 = np.ma.masked_where(menorqdesvv + menorqdesvv_k2 == -1, menorqdesvv_k2)
# fim do 3º passo

        maxsijk = np.where(abs(slices_lv[sc] - slices_mv[sc]) > abs(slices_lv_k2[sc] - slices_mv_k2[sc]),
                        abs(slices_lv[sc] - slices_mv[sc]), abs(slices_lv_k2[sc] - slices_mv_k2[sc])) #criando a matriz que apresenta o Sijk máximo entre t1 e t2

        sijk1 = abs(filtro_lv_k1 - slices_mv) #criando o Sijk apenas para os neighbours
        sijk1_filt = np.ma.masked_where(sijk1 > maxsijk + 0.01, filtro_lv_k1) #aplicando a inequação para o filtro
        sijk2 = abs(filtro_lv_k2 - slices_mv_k2)
        sijk2_filt = np.ma.masked_where(sijk2 > maxsijk + 0.01, filtro_lv_k2)

        sijk1_final = np.ma.masked_where(sijk1_filt + sijk2_filt == -1, sijk1_filt) #selecionando apenas os pixels que obedecem a inequação em t1 e t2

        maxtijk = np.where(abs(slices_mv[sc] - slices_mv_ko[sc]) > abs(slices_mv_k2[sc] - slices_mv_ko[sc]),
                        abs(slices_mv[sc] - slices_mv_ko[sc]), abs(slices_mv_k2[sc] - slices_mv_ko[sc])) #repetindo o processo p/ Tijk

        filtro_mv_ko = np.ma.masked_where(slices_mv_ko + sijk1_final == -1, slices_mv_ko)

        tijk1 = abs(slices_mv - filtro_mv_ko)
        tijk1_filt = np.ma.masked_where(tijk1 > maxtijk + 0.01, slices_mv)
        tijk2 = abs(slices_mv_k2 - filtro_mv_ko)
        tijk2_filt = np.ma.masked_where(tijk2 > maxtijk + 0.01, slices_mv_k2)

        tijk1_final = np.ma.masked_where(tijk1_filt + tijk2_filt == -1, tijk1_filt)

        neighbours = np.ma.masked_where(tijk1_final + sijk1_final == -1, sijk1_final) #definindo os neighbours segundo o Sijk e o Tijk
# fim do 4º passo para o v
#repetindo os passoas para o infravermelho do Landsat e MODIS
        slices_liv = make_slices(in_data_liv, (mw, mw))
        slices_miv = make_slices(in_data_miv, (mw, mw))
        slices_liv_k2 = make_slices(in_data_liv_k2, (mw, mw))
        slices_miv_k2 = make_slices(in_data_miv_k2, (mw, mw))
        slices_miv_ko = make_slices(in_data_miv_ko, (mw, mw))

        difabsiv = [abs(valoriv - slices_liv[sc]) for valoriv in slices_liv]
        desvpadiv = np.std(np.ma.dstack(slices_liv), 2)
        menorqdesviv = np.ma.masked_where(difabsiv > desvpadiv, slices_liv)

        difabsiv_k2 = [abs(valoriv - slices_liv_k2[sc]) for valoriv in slices_liv_k2]
        desvpadiv_k2 = np.std(np.ma.dstack(slices_liv_k2), 2)
        menorqdesviv_k2 = np.ma.masked_where(difabsiv_k2 > desvpadiv_k2, slices_liv_k2)

        filtro_liv_k1 = np.ma.masked_where(menorqdesviv + menorqdesviv_k2 == -1, menorqdesviv)
        filtro_liv_k2 = np.ma.masked_where(menorqdesviv + menorqdesviv_k2 == -1, menorqdesviv_k2)
        # fim do 3º passo para o iv

        maxsijkiv = np.where(abs(slices_liv[sc] - slices_miv[sc]) > abs(slices_liv_k2[sc] - slices_miv_k2[sc]),
                           abs(slices_liv[sc] - slices_miv[sc]), abs(slices_liv_k2[sc] - slices_miv_k2[sc]))

        sijk1iv = abs(filtro_liv_k1 - slices_miv)
        sijk1iv_filt = np.ma.masked_where(sijk1iv > maxsijkiv + 0.015, filtro_liv_k1)
        sijk2iv = abs(filtro_liv_k2 - slices_miv_k2)
        sijk2iv_filt = np.ma.masked_where(sijk2iv > maxsijkiv + 0.015, filtro_liv_k2)

        sijk1iv_final = np.ma.masked_where(sijk1iv_filt + sijk2iv_filt == -1, sijk1iv_filt)

        maxtijkiv = np.where(abs(slices_miv[sc] - slices_miv_ko[sc]) > abs(slices_miv_k2[sc] - slices_miv_ko[sc]),
                           abs(slices_miv[sc] - slices_miv_ko[sc]), abs(slices_miv_k2[sc] - slices_miv_ko[sc]))

        filtro_miv_ko = np.ma.masked_where(slices_miv_ko + sijk1iv_final == -1, slices_miv_ko)

        tijk1iv = abs(slices_miv - filtro_miv_ko)
        tijk1iv_filt = np.ma.masked_where(tijk1iv > maxtijkiv + 0.015, slices_miv)
        tijk2iv = abs(slices_miv_k2 - filtro_miv_ko)
        tijk2iv_filt = np.ma.masked_where(tijk2iv > maxtijkiv + 0.015, slices_miv_k2)

        tijk1iv_final = np.ma.masked_where(tijk1iv_filt + tijk2iv_filt == -1, tijk1iv_filt)

        neighboursiv = np.ma.masked_where(tijk1iv_final + sijk1iv_final == -1, sijk1iv_final)
# fim do 4º passo para o iv

        neighbours = np.ma.masked_where(neighbours + neighboursiv == -1, neighbours) #definindo os neighbours segundo o V e o IV
        neighbours = neighbours.filled(-99)

        out_data_lv = np.ones(in_data_lv.shape, np.float32) * -99 #criando uma matriz no formato da matriz lida (chunk)
        for s in range(0, ns): #criando um loop entre as camadas do neighbours
            out_data_lv[lsp:-lsp, lsp:-lsp] = neighbours[s] #pegando a 1ª matriz da lista neighbours e inserindo na matriz recém criada
            y = (s + 1) // (mw + 1) #calculando a coordenada-y de cada pixel da camada-s atual
            x = s + 1 - (y * mw) - 1 #calculando a coordenada-x de cada pixel da camada-s atual
            D = 1 + ((np.sqrt(np.square(x - lsp) + np.square(y - lsp))) * 30) / 250 #calculando D de cada pixel da camada-s atual
            d_filt = np.where(out_data_lv > 0, D, -99) #atribuindo valor "D" para os neighbours

            out_data_lv_rec = np.delete(d_filt, np.s_[:lsp], 0) #excluindo as 24 primeiras linhas (de 0 a 23)
            out_filtrolv = out_ds_lv.GetRasterBand(s + 1) #pegando a camada da imagem de saída
            out_filtrolv.SetNoDataValue(-99)

            out_filtrolv.WriteArray(out_data_lv_rec[0:n], 0, 0) #colando as 2 primeiras linhas no início da imagem de saída

        out_filtrolv.FlushCache()
        out_filtrolv.ComputeStatistics(False)

        print('imagem {} criada'.format(nome))

        del in_data_lv, in_data_mv, in_data_lv_k2, in_data_mv_k2, in_data_mv_ko, out_ds_lv, slices_lv, slices_mv, slices_lv_k2, slices_mv_k2,
         slices_mv_ko, difabsv, desvpadv, menorqdesvv, difabsv_k2, desvpadv_k2, menorqdesvv_k2, filtro_lv_k1, filtro_lv_k2, maxsijk, sijk1,
          sijk1_filt, sijk2, sijk2_filt, sijk1_final, maxtijk, filtro_mv_ko, tijk1, tijk1_filt, tijk2, tijk2_filt, tijk1_final, neighbours,
           out_data_lv, out_filtrolv, slices_liv, slices_miv, slices_liv_k2, slices_miv_k2, slices_miv_ko, difabsiv, desvpadiv, menorqdesviv,
            difabsiv_k2, desvpadiv_k2, menorqdesviv_k2, filtro_liv_k1, filtro_liv_k2, maxsijkiv, sijk1iv, sijk1iv_filt, sijk2iv, sijk2iv_filt,
             sijk1iv_final, maxtijkiv, filtro_miv_ko, tijk1iv, tijk1iv_filt, tijk2iv, tijk2iv_filt, tijk1iv_final, neighboursiv, in_data_liv,
              in_data_miv, in_data_liv_k2, in_data_miv_k2, in_data_miv_ko, out_data_lv_rec

        break

    q = len(img_de_saida)
    if q == 0:
        break
    else:
        img_de_saida.pop(0)

del in_ds

# STARFM PART II

In [ ]:
os.chdir(r"/content/drive/MyDrive/colab_imagens")

imagem_lv = 'reflectance_2016_10_29.tif' # Load the first (Albedo, surface reflectance or Ts) LANDSAT image (the day before the predicted date)
in_ds = gdal.Open(imagem_lv)
in_band_lv = in_ds.GetRasterBand(4)

imagem_mv = 'modis_gq2016_10_29.tif' # Load the first (Albedo, surface reflectance or Ts) MODIS image (the day before the predicted date)
in_ds_mv = gdal.Open(imagem_mv)
in_band_mv = in_ds_mv.GetRasterBand(2)

imagem_lv_k2 = 'reflectance_2017_01_17.tif' # Load the second (Albedo, surface reflectance or Ts) LANDSAT image (the day after the predicted date)
in_ds_k2 = gdal.Open(imagem_lv_k2)
in_band_lv_k2 = in_ds_k2.GetRasterBand(4)

imagem_mv_k2 = 'modis_gq2017_01_17.tif' # Load the second (Albedo, surface reflectance or Ts) MODIS image (the day after the predicted date)
in_ds_mv_k2 = gdal.Open(imagem_mv_k2)
in_band_mv_k2 = in_ds_mv_k2.GetRasterBand(2)

imagem_mv_ko = 'modis_gq2016_12_31.tif' # Load the third (Albedo, surface reflectance or Ts) MODIS image (predicted date)
in_ds_mv_ko = gdal.Open(imagem_mv_ko)
in_band_mv_ko = in_ds_mv_ko.GetRasterBand(2)

xsize = in_band_lv.XSize #número de colunas
ysize = in_band_lv.YSize #número de linhas

print(ysize)

neighbours = []
for z in range(1, ysize+1):
  neighbours.append('n{}.tif'.format(z))

mw = 49 #moving window
lp = mw - 1 #linhas perdidas
ns = mw * mw #número de slices
lsp = int(lp / 2) #linhas superiores perdidas --------------------------------------------ALTERAR LINHAS 77-80, 98 e 100
sc = ns // 2 #slice central
n = 6 #número de linhas a serem resolvidas

for i in range(0, ysize, n):
    if n + lp + i < ysize:
        rows = n + lp
    else:
        rows = ysize - i
        if rows < mw:
            break

    in_data_lv = in_band_lv.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_mv = in_band_mv.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_lv_k2 = in_band_lv_k2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_mv_k2 = in_band_mv_k2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    in_data_mv_ko = in_band_mv_ko.ReadAsArray(0, i, xsize, rows).astype(np.float32)

    slices_lv = make_slices(in_data_lv, (mw, mw))
    slices_mv = make_slices(in_data_mv, (mw, mw))
    slices_lv_k2 = make_slices(in_data_lv_k2, (mw, mw))
    slices_mv_k2 = make_slices(in_data_mv_k2, (mw, mw))
    slices_mv_ko = make_slices(in_data_mv_ko, (mw, mw))

    sijk1 = abs(np.ma.dstack(slices_lv) - np.ma.dstack(slices_mv)) #calculando Sijk para T1
    sijk2 = abs(np.ma.dstack(slices_lv_k2) - np.ma.dstack(slices_mv_k2)) #calculando Sijk para T2

    tijk1 = abs(np.ma.dstack(slices_mv) - np.ma.dstack(slices_mv_ko)) #calculando Tijk para T1
    tijk2 = abs(np.ma.dstack(slices_mv_k2) - np.ma.dstack(slices_mv_ko)) #calculando Tijk para T2
    ysize_teste = sijk1.shape
    print('Esse é o número de linhas de Sijk1 {}'.format(ysize_teste))
    for nome in neighbours:
        in_ds_nome = gdal.Open(nome) #lendo as imagens que possuem as informações de "D" nos pixels que são neighbours
        lendo = in_ds_nome.ReadAsArray().astype(np.float32) #ao inves de ler uma banda da img de entrada, lê-se todas
        lendo = np.ma.masked_where(lendo == -99, lendo) #mascarando os valores dos pixels que não são neighbours

        s1 = np.ones(lendo.shape, np.float32) * -99 #criando uma matriz no formato da imagem lida
        s2 = np.ones(lendo.shape, np.float32) * -99
        t1 = np.ones(lendo.shape, np.float32) * -99
        t2 = np.ones(lendo.shape, np.float32) * -99

        for c in range(0, ns): #criando um loop entre o nº de matrizes das listas Sijk e Tijk
          if n + lp + i < ysize:
            s1[c, :, lsp:-lsp] = sijk1[:, :, c] #pegando o sijk1 (lista empilhada(y, x, z)) e colocando na matriz (z, y, x)
            s2[c, :, lsp:-lsp] = sijk2[:, :, c]
            t1[c, :, lsp:-lsp] = tijk1[:, :, c]
            t2[c, :, lsp:-lsp] = tijk2[:, :, c]
          else:
            lin = int(ysize - i - lp)
            s1[c, :lin, lsp:-lsp] = sijk1[:, :, c] #pegando o sijk1 (lista empilhada(y, x, z)) e colocando na matriz (z, y, x)
            s2[c, :lin, lsp:-lsp] = sijk2[:, :, c]
            t1[c, :lin, lsp:-lsp] = tijk1[:, :, c]
            t2[c, :lin, lsp:-lsp] = tijk2[:, :, c]

        Cijk1 = lendo * np.log((s1 * 10000) + 1) * np.log((t1 * 10000) + 1)
        Cijk2 = lendo * np.log((s2 * 10000) + 1) * np.log((t2 * 10000) + 1)
        divisao1 = 1 / Cijk1
        divisao2 = 1 / Cijk2


        Wijk1 = divisao1 / (divisao1.sum(0) + divisao2.sum(0)) #na função .sum(0) a soma é na vertical (a matriz numpy tem formato z, y, x)
        Wijk2 = divisao2 / (divisao1.sum(0) + divisao2.sum(0)) #(z, y, x) -> (0, 1, 2), por isso usa-se .sum(0)

        Wijk1 = Wijk1.filled(0) #para que os pixels que não são neighbours não participem do cálculo de "L", substiu-se os mesmos por 0
        Wijk2 = Wijk2.filled(0)

        M_L_M_1= slices_mv_ko[sc] + slices_lv[sc] - slices_mv[sc]      #M(xw/2, yw/2, to) + L(xw/2, yw/2, tk1) - M(xw/2, yw/2, tk1)
        M_L_M_2 = slices_mv_ko[sc] + slices_lv_k2[sc] - slices_mv_k2[sc]#M(xw/2, yw/2, to) + L(xw/2, yw/2, tk2) - M(xw/2, yw/2, tk2)
        
        temporal_k0_igual = np.ones((n, xsize), np.float) * -99
        spectral_k0_igual = np.ones((n, xsize), np.float) * -99
        temporal_k2_igual = np.ones((n, xsize), np.float) * -99
        spectral_k2_igual = np.ones((n, xsize), np.float) * -99
        
        if n + lp + i < ysize:
          temporal_k0_igual[:, lsp:-lsp] = slices_mv_ko[sc] - slices_mv[sc]
          spectral_k0_igual[:, lsp:-lsp] = slices_lv[sc] - slices_mv[sc]
          temporal_k2_igual[:, lsp:-lsp] = slices_mv_ko[sc] - slices_mv_k2[sc]
          spectral_k2_igual[:, lsp:-lsp] = slices_lv_k2[sc] - slices_mv_k2[sc]
        else:
          temporal_k0_igual[:2, lsp:-lsp] = slices_mv_ko[sc] - slices_mv[sc]
          spectral_k0_igual[:2, lsp:-lsp] = slices_lv[sc] - slices_mv[sc]
          temporal_k2_igual[:2, lsp:-lsp] = slices_mv_ko[sc] - slices_mv_k2[sc]
          spectral_k2_igual[:2, lsp:-lsp] = slices_lv_k2[sc] - slices_mv_k2[sc]

        if n + lp + i < ysize:
          T1 = np.ones((n, xsize), np.float) * -99
          T2 = np.ones((n, xsize), np.float) * -99
          T1[:, lsp:-lsp] = M_L_M_1[:, :]             #pegando o M_L_M_1 e colocando na matriz do numpy
          T2[:, lsp:-lsp] = M_L_M_2[:, :]             #pegando o M_L_M_2 e colocando na matriz do numpy       
        else:
          T1 = np.ones((6, xsize), np.float) * -99
          T2 = np.ones((6, xsize), np.float) * -99  
          T1[:2, lsp:-lsp] = M_L_M_1[:, :]             #pegando o M_L_M_1 e colocando na matriz do numpy
          T2[:2, lsp:-lsp] = M_L_M_2[:, :]             #pegando o M_L_M_2 e colocando na matriz do numpy
        
        if n + lp + i < ysize:
          L = np.zeros((n, xsize), np.float) #criando uma matriz com valores 0, 2 linhas e xsize colunas
        else:
          L = np.zeros((6, xsize), np.float) 
        for m in range(0, ns): #criando um loop dentro do nº de camadas do Wijk
            L = L + ((Wijk1[m] * T1) + (Wijk2[m] * T2)) #L inicialmente é 0, mas vai sendo substituído ao passo em que as somas vão se realizando

        L = np.where(temporal_k0_igual == 0.0, T1, L)# Filtro p/ quando não há diferença entre imgs MODIS (01 e 00)
        L = np.where(spectral_k0_igual == 0.0, T1, L)# Filtro p/ quando não há diferença entre imgs L-M (01)
        L = np.where(temporal_k2_igual == 0.0, T2, L)# Filtro p/ quando não há diferença entre imgs MODIS (02 e 00)
        L = np.where(spectral_k2_igual == 0.0, T2, L)# Filtro p/ quando não há diferença entre imgs L-M (02)

        driver = gdal.GetDriverByName('GTiff')
        out_ds = driver.Create('L{}.tif'.format(i), xsize, n, 1, gdal.GDT_Float32) #criando a imagem de saída com xsize colunas, 2 linhas e 1 camada
        out_ds.SetProjection(in_ds_nome.GetProjection())
        out_ds.SetGeoTransform(in_ds_nome.GetGeoTransform())
        out = out_ds.GetRasterBand(1)
        out.SetNoDataValue(-99)

        out.WriteArray(L[0:n], 0, 0) #escrevendo as duas primeiras (e únicas) linhas de L no início da imagem de saída

        out.FlushCache()
        out.ComputeStatistics(False)

        print('imagem L{}.tif criada'.format(i))

        del in_data_lv
        del in_data_mv
        del in_data_lv_k2
        del in_data_mv_k2
        del in_data_mv_ko
        del slices_lv
        del slices_mv
        del slices_lv_k2
        del slices_mv_k2
        del slices_mv_ko
        del in_ds_nome
        del lendo
        del s1
        del s2
        del t1
        del t2
        del Cijk1
        del Cijk2
        del Wijk1
        del Wijk2
        del M_L_M_1
        del M_L_M_2
        del T1
        del T2
        del out
        del out_ds

        break

    q = len(neighbours)
    if q == 0:
        break
    else:
        neighbours.pop(0)

del in_ds
del in_ds_mv
del in_ds_k2
del in_ds_mv_k2
del in_ds_mv_ko

182
Esse é o número de linhas de Sijk1 (6, 181, 2401)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in log


imagem L0.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L6.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L12.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L18.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L24.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L30.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L36.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L42.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L48.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L54.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L60.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L66.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L72.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 2401)
imagem L78.tif criada
Esse é o número de linhas de Sijk1 (6, 181, 

# Merging raster data

In [ ]:
# Merge raster data produced after the STARFM execution

# What is the name of the output image?
var = 'ref_iv_' #examples: albedo, ref_iv, ref_verm and tsup
date = '2016_12_31' #y-m-d
name = var + date

import glob

# list all files in directory that match pattern
List = glob.glob("L*.tif")
print(List)

# build virtual raster and convert to geotiff 'n{}.tif'.format(z)
vrt = gdal.BuildVRT("merged.vrt", List)
gdal.Translate("{}.tif".format(name), vrt, xRes = 30, yRes = -30)
vrt = None

['L0.tif', 'L6.tif', 'L12.tif', 'L18.tif', 'L24.tif', 'L30.tif', 'L36.tif', 'L42.tif', 'L48.tif', 'L54.tif', 'L60.tif', 'L66.tif', 'L72.tif', 'L78.tif', 'L84.tif', 'L90.tif', 'L96.tif', 'L102.tif', 'L108.tif', 'L114.tif', 'L120.tif', 'L126.tif', 'L132.tif']


# Exporting land use and land cover image

In [ ]:
data1 = ee.String('2016')
imagem_1 = ee.String('mapbiomas_')
saida_1 = imagem_1.cat(data1)

#area = ee.Geometry.Polygon(
#        [[[-40.43748032601223, -9.375816954123401],
#          [-40.43748032601223, -9.411721142856367],
#          [-40.38821351082668, -9.411721142856367],
#          [-40.38821351082668, -9.375816954123401]]])
area = ee.Geometry.Polygon(
        [[[-40.443765656859135, -9.369681936878271],
          [-40.443765656859135, -9.4184573955032],
          [-40.38179589977906, -9.4184573955032],
          [-40.38179589977906, -9.369681936878271]]]);

#img = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2015-08-24', '2015-08-25')

mapbiomas = ee.Image('projects/mapbiomas-workspace/public/collection5/mapbiomas_collection50_integration_v1').select(['classification_2016'])

task1 = ee.batch.Export.image.toDrive(image=mapbiomas, folder='colab_imagens', description='Mapbiomas', scale=30, region=area, fileNamePrefix=saida_1.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task1.start()
task1.status()

# SEBAL SCRIPT

In [ ]:
area = ee.Geometry.Polygon(
        [[[-40.43748032601223, -9.375816954123401],
          [-40.43748032601223, -9.411721142856367],
          [-40.38821351082668, -9.411721142856367],
          [-40.38821351082668, -9.375816954123401]]])
 
os.chdir(r"/content/drive/MyDrive/colab_imagens")
# Albedo is the image used to GetProjection(), GetGeoTransform(), XSize and YSize
albedo_open = gdal.Open('albedo_2016_12_31.tif')
albedo_in_band = albedo_open.GetRasterBand(1)
albedo_in_data = albedo_in_band.ReadAsArray().astype(np.float)
 
NIR_open = gdal.Open('ref_iv_2016_12_31.tif')#.GetRasterBand(1).ReadAsArray().astype(np.float)# Near Infra-red Image
NIR_in_band = NIR_open.GetRasterBand(1)
NIR_in_data = NIR_in_band.ReadAsArray().astype(np.float)
 
RED_open = gdal.Open('ref_verm_2016_12_31.tif')#.GetRasterBand(1).ReadAsArray().astype(np.float)# Red image
RED_in_band = RED_open.GetRasterBand(1)
RED_in_data = RED_in_band.ReadAsArray().astype(np.float)
 
tsup_open = gdal.Open('tsup_2016_12_31.tif')#.GetRasterBand(1).ReadAsArray().astype(np.float)# Surface Temperature Image
tsup_in_band = tsup_open.GetRasterBand(1)
tsup_in_data = tsup_in_band.ReadAsArray().astype(np.float)
 
mapbiomas_open = gdal.Open('mapbiomas_2016.tif')#.GetRasterBand(1).ReadAsArray().astype(np.int)# Land use and land cover Image
mapbiomas_in_band = mapbiomas_open.GetRasterBand(1)
mapbiomas = mapbiomas_in_band.ReadAsArray().astype(np.float)
mapbiomas_in_data = np.ones(albedo_in_data.shape, np.float32) * -99
#lsp = int(lp / 2) #linhas/colunas perdidas durante a execução do STARFM (24)
mapbiomas_in_data[:-4, 24:-24] = mapbiomas[24:-24, 24:-24]
 
albedo = np.ma.masked_where(albedo_in_data < 0, albedo_in_data)
albedo = np.ma.masked_where(albedo > 1, albedo)
 
NIR = np.where(NIR_in_data < 0.0, 0.0, NIR_in_data)
NIR = np.ma.masked_where(NIR < 0, NIR)
 
RED = np.where(RED_in_data < 0.0, 0.0, RED_in_data)
RED = np.ma.masked_where(RED < 0, RED)
 
tsup = np.ma.masked_where(tsup_in_data < 273, tsup_in_data)
tsup = np.ma.masked_where(tsup > 340, tsup)
 
lat = -9.357#///////// Inserir a Latitude média do RECORTE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 
ta = 34.7#/////////// Inserir A Ta (°C) no instante da passagem do satélite!!!!!!!!!!!!!!!!!!!!
ur = 29.62#/////////// Inserir A UR (%) no instante da passagem do satélite!!!!!!!!!!!!!!!!!!!!!
pa = 96.802#/////////// Inserir A Pa (kPa) no instante da passagem do satélite!!!!!!!!!!!!!!!!!!!
 
u = 3.062#//////////// Inserir a Vel do vento (m/s, a 10m) no instante da passagem do satélite!!
rs_24 = 26.3#//////// Inserir A Radiação global diária (MJ)!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 
precipitation60 = 45.2# //Inserir a precipitação dos últimos 60 dias!!!!!!!!!!!!
ETr60 = 381.9#///////////Inserir a ETref dos últimos 60 dias!!!!!!!!!!!!!!!!!!!
 
ETref_00 = 4.944755653
ETref_periodo_00 = 12.14
 
DOA = 366#//////////// Inserir o Dia de Ordem do Ano!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
data = '2016_12_31_'#// Inserir A DATA (ANO-MÊS-DIA)!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 
ang_zen = ee.Image('MODIS/006/MYD09GA/2016_12_31')# // Inserir A DATA (ANO-MÊS-DIA)!!!!!!!!!!!!!!!!
 
#EXPORTS#####################################################
nome_ndvi = 'ndvi.tif'
export_ndvi = data + nome_ndvi
 
nome_et24 = 'et_24.tif'
export_et24 = data + nome_et24
 
#CALCULUS####################################################
Tfac = 2.6 - (13*(precipitation60/ETr60))
print(Tfac)
 
#//calculando o dr e Cos(z)
argumento = 2*np.pi*(DOA-1)/365
dr = 1.000110+0.034221*np.cos(argumento)+0.001280*np.sin(argumento)+0.000719*np.cos(2*argumento)+0.000077*np.sin(2*argumento)
print ('dr é: ', dr)
 
cos_z_img = ang_zen.select(['SolarZenith']).multiply(0.01).multiply(np.pi).divide(180).cos()
 
cos_z_red = cos_z_img.reduceRegion(
  ee.Reducer.mean(),
  area,
  30)
 
#print(cos_z_red)
 
cos_z = cos_z_red.getInfo()['SolarZenith']
 
elevacao = ((np.pi/2) - math.acos(cos_z))*180/np.pi
 
print ('Cos(z) é: ', cos_z)
print ('elevacao é: ', elevacao)
 
#//calculando o SAVI
savi = (1.1 * (NIR - RED)) / (0.1 + NIR + RED)
#print ('O SAVI é: ', savi)
 
#//calculando o IAF
iaf = np.where(savi < 0.688000, np.log((0.69 - savi) / 0.59) * (-1) / 0.91, 6)
iafcorrigido = np.where(savi < 0.000001, 0, iaf)
 
#// calculando e_o
e_o = np.where(iafcorrigido > 2.999999, 0.98, 0.95 + (0.01 * iafcorrigido))
e_ocorrigido = np.where(iafcorrigido < 0.000001, 0.985, e_o)
 
#// calculando e_nb
e_nb = np.where(iafcorrigido > 2.999999, 0.98, 0.97 + (0.0033 * iafcorrigido))
e_nbcorrigido = np.where(iafcorrigido < 0.000001, 0.99, e_nb)
 
#// calculando Rol,emi
rol_emi = e_o * 0.0000000567 * tsup**4
 
#// calculando a transmitância
e_a = 0.61078 * (np.exp((17.269 * ta)/(237.3 + ta))) * ur/100
w = (10 * (0.14 * 10 * e_a * (pa/101))) + 0.21
transmitancia = 0.35 + 0.627 * (np.exp((0.00146*(-1)*pa/cos_z) - 0.075 * ((w/cos_z)**0.4)))
 
print ('e_a equivale a : ', e_a)
print ('w equivale a : ', w)
print ('transmitancia equivale a : ', transmitancia)
 
#//calculando a Radiação global (Allen et al., 2007)
roc = 1367 * dr * cos_z * transmitancia
print ('a radiação global é: ', roc)
 
#// calculando emissividade atm
e_atm = 0.625 * (((1000 * e_a) / (273.15 + ta)) ** 0.131)
 
#// calculando Rol,atm
rol_atm = e_atm * 0.0000000567 * (ta + 273.15)**4
print ('a radiação atm é: ', rol_atm)
 
#// calculando Rn
rn = (1 - albedo) * roc - rol_emi + e_ocorrigido * rol_atm 
 
#// calculando NDVI e G
ndvi = (NIR - RED) / (NIR + RED)
 
#print ('o NDVI é: ', ndvi)
 
g_corrigido = np.where(ndvi < 0.05, 0.3 * rn, ((tsup - 273.15) * (0.0038 + 0.0074 * albedo) * (1 - 0.98 * (ndvi ** 4))) * rn)
 
#//#//#//#//#//#//#//#//#//#// BALANÇO DE ENERGIA#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
#//PIXEL FRIO CIMEC
 
p95 = np.percentile(ndvi[ndvi > 0], 95)
print('Esse é o percentil 95 do NDVI ', p95)
 
f1_frio_0 = np.ma.masked_where(mapbiomas_in_data != 36, ndvi)
f1_frio = np.ma.masked_where(f1_frio_0 < p95, tsup)
 
p20 =  np.percentile(f1_frio[f1_frio > 0], 20)
print('esse é o percentil 20 da Tsup: ', p20)
 
f2_frio = np.ma.masked_where(f1_frio > p20, f1_frio)
 
f2_frio_med = np.mean(f2_frio[f2_frio > 0])
 
f3_frio = np.ma.masked_where((f2_frio_med + 0.2) < f2_frio, f2_frio)
f3_frio = np.ma.masked_where(f2_frio < (f2_frio_med - 0.2), f3_frio)
 
albedo_chave = 0.001343 * elevacao + (0.3281 * np.exp((-0.0188) * elevacao))
print('albedo chave é: ', albedo_chave)
 
f4_frio = np.ma.masked_where((albedo_chave + 0.02) < albedo, f3_frio)
f4_frio = np.ma.masked_where(albedo < (albedo_chave - 0.02), f4_frio)
 
p_frio_ok = f4_frio.min()
 
print('Esse é o pixel frio do CIMEC: ', p_frio_ok)
 
#//PIXEL QUENTE CIMEC
p5 = np.percentile(ndvi[ndvi>0], 5)
 
f1_quente = np.ma.masked_where(ndvi > p5, tsup)
 
p80 = np.percentile(f1_quente[f1_quente > 0], 80)
 
f2_quente = np.ma.masked_where(f1_quente < p80, f1_quente)
 
f2_quente_med = np.mean(f2_quente)
 
f2_quente_med_2 = f2_quente_med - Tfac
 
f3_quente = np.ma.masked_where(f2_quente > (f2_quente_med_2 + 3.0), f2_quente)
f3_quente = np.ma.masked_where(f3_quente < (f2_quente_med_2 - 3.0), f3_quente)
#// ESSE nº de corte do f3_quente pode precisar de ajuste, usar 2.0 ou 3.5 pode resolver...
 
p_quente_ok = f3_quente.max()
 
print('Esse é o pixel quente do CIMEC: ', p_quente_ok)
 
#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
 
#//SAVI do pixel quente
savi_pq = np.ma.masked_where(ndvi > p5, tsup)
savi_pq = np.ma.masked_where(savi_pq != p_quente_ok, savi)
savi_quente = savi_pq.max()
print ('O savi do Pixel quente é: ', savi_quente)
 
#//Rn do pixel quente
rn_pq = np.ma.masked_where(ndvi > p5, tsup)
rn_pq = np.ma.masked_where(rn_pq != p_quente_ok, rn)
rn_quente = rn_pq.max()
print ('O Rn do Pixel quente é: ', rn_quente)
 
#//G do pixel quente
g_pq = np.ma.masked_where(ndvi > p5, tsup)
g_pq = np.ma.masked_where(g_pq != p_quente_ok, g_corrigido)
g_quente = g_pq.max()
print ('O G do Pixel quente é: ', g_quente)
 
#//calculando os coeficientes da regressão linear dT = a + bT
 
u_estrela1 = u * 0.41/np.log(10/0.024)
u200 = u_estrela1 * np.log(200/0.024)/0.41
zom = np.exp(-5.809+5.62*savi_quente)
u_estrela = (u200*0.41)/np.log(200/zom)
rah = (np.log(2/0.1))/(0.41 * u_estrela)
b = (rn_quente - g_quente)*rah / (1.15*1004*(p_quente_ok - p_frio_ok))
a = (-1)*b*(p_frio_ok - 273.15)
dT = a + b *(p_quente_ok - 273.15)
H = 1.15 * 1004 * (a + (b * (p_quente_ok - 273.15)))/rah
 
print ('O Fluxo de calor sensível H é: ', H)
rn_g = rn_quente - g_quente
print ('Rn - G é: ', rn_g)
#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//calculando nas imagens#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//
zom_img = np.exp(-5.809+5.62*savi)
u_estrela_img = (u200*0.41)/np.log(200/zom_img)
rah_img = (np.log(2/0.1))/(0.41 * u_estrela_img)
H_img = 1.15 * 1004 * (a + (b * (tsup - 273.15)))/rah_img
 
#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//INICIO DO PROCESSO ITERATIVO#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//1ª CORREÇÃO#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
L = (-1)*1.15*1004*(u_estrela**3)*p_quente_ok/(0.41*9.81*H)
 
x_0_1m = ((1-16)*0.1/L)**0.25
x_2m = ((1-16)*2/L)**0.25
x_200m = ((1-16)*200/L)**0.25
 
psi_h0_1 = 2*np.log((1+(x_0_1m**2))/2)
psi_h2 = 2*np.log((1+(x_2m**2))/2)
psi_m200 = 2*np.log((1+x_200m)/2)+np.log((1+x_200m**2)/2)-2*np.arctan(x_200m)+0.5*np.pi
 
u_estrela_corr_1 = u200*0.41/((np.log(200/zom))-psi_m200)
rah_corr_1 = (np.log(2/0.1)-psi_h2+psi_h0_1)/(0.41*u_estrela_corr_1)
b_corr = (rn_quente - g_quente)*rah_corr_1 / (1.15*1004*(p_quente_ok - p_frio_ok))
a_corr = (-1)*b_corr*(p_frio_ok - 273.15)
dT_corr = a_corr + b_corr *(p_quente_ok - 273.15)
H = 1.15 * 1004 * (a_corr + (b_corr * (p_quente_ok - 273.15)))/rah_corr_1
 
#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//calculando 1ª CORREÇÃO nas imagens #//#//#//#//#//#//#//#//#//#//#//#//#//#//#//
L_img = (-1)*1.15*1004*(u_estrela_img**3)*tsup/(0.41*9.81*H_img)
x_0_1m_img = ((1-16)*0.1/L_img)**0.25
x_2m_img = ((1-16)*2/L_img)**0.25
x_200m_img = ((1-16)*200/L_img)**0.25
 
psi_h0_1_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_0_1m_img**2))/2), (-1)*5*(0.1/L_img))
psi_h2_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_2m_img**2))/2), (-1)*5*(2/L_img))
psi_m200_img_c = np.where(L_img < 0.0, 2*np.log((1+x_200m_img)/2)+np.log((1+(x_200m_img**2))/2)-(2*np.arctan(x_200m_img))+0.5*(np.pi),
                          (-1)*5*(200/L_img))
u_estrela_corr_img_1 = (u200*0.41)/((np.log(200/zom_img))-psi_m200_img_c)
rah_corr_img_1 = (np.log(2/0.1)-psi_h2_img_c+psi_h0_1_img_c)/(0.41 * u_estrela_corr_img_1)
H_corr_img = 1.15 * 1004 * (a_corr + (b_corr * (tsup - 273.15)))/rah_corr_img_1
 
err_rel = abs(rah_corr_1 - rah)*100/rah
print('a vale: ', a_corr, 'b vale: ', b_corr, 'O Erro relativo é: ', err_rel)
print('H é: ', H)
 
lista_u_star = []
lista_rah = []
lista_u_star_img = []
lista_rah_img = []
 
lista_u_star.append(u_estrela_corr_1)
lista_rah.append(rah_corr_1)
lista_u_star_img.append(u_estrela_corr_img_1)
lista_rah_img.append(rah_corr_img_1)
 
 
rodadas = 1
 
while err_rel > 3:
    rodadas += 1
    lista_u_star.append('u_estrela_corr_{}'.format(rodadas))
    lista_rah.append('rah_corr_{}'.format(rodadas))
    lista_u_star_img.append('u_estrela_corr_img_{}'.format(rodadas))    
    lista_rah_img.append('rah_corr_img_{}'.format(rodadas))
 
    L = (-1)*1.15*1004*((lista_u_star[0])**3)*p_quente_ok/(0.41*9.81*H)
 
    x_0_1m = ((1-16)*0.1/L)**0.25
    x_2m = ((1-16)*2/L)**0.25
    x_200m = ((1-16)*200/L)**0.25
 
    psi_h0_1 = 2*np.log((1+(x_0_1m**2))/2)
    psi_h2 = 2*np.log((1+(x_2m**2))/2)
    psi_m200 = 2*np.log((1+x_200m)/2)+np.log((1+x_200m**2)/2)-2*np.arctan(x_200m)+0.5*np.pi
 
    lista_u_star[1] = u200*0.41/((np.log(200/zom))-psi_m200)
    lista_rah[1] = (np.log(2/0.1)-psi_h2+psi_h0_1)/(0.41*(lista_u_star[1]))
    b_corr = (rn_quente - g_quente)*lista_rah[1] / (1.15*1004*(p_quente_ok - p_frio_ok))
    a_corr = (-1)*b_corr*(p_frio_ok - 273.15)
 
    H = 1.15 * 1004 * (a_corr + (b_corr * (p_quente_ok - 273.15)))/lista_rah[1]
 
    #calculando nas imagens:
    L_img = (-1)*1.15*1004*(((lista_u_star_img[0]))**3)*tsup/(0.41*9.81*H_corr_img)
    x_0_1m_img = ((1-16)*0.1/L_img)**0.25
    x_2m_img = ((1-16)*2/L_img)**0.25
    x_200m_img = ((1-16)*200/L_img)**0.25
 
    psi_h0_1_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_0_1m_img**2))/2), (-1)*5*(0.1/L_img))
    psi_h2_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_2m_img**2))/2), (-1)*5*(2/L_img))
    psi_m200_img_c = np.where(L_img < 0.0, 2*np.log((1+x_200m_img)/2)+np.log((1+(x_200m_img**2))/2)-(2*np.arctan(x_200m_img))+0.5*(np.pi),
                              (-1)*5*(200/L_img))
    lista_u_star_img[1] = (u200*0.41)/((np.log(200/zom_img))-psi_m200_img_c)
    lista_rah_img[1] = (np.log(2/0.1)-psi_h2_img_c+psi_h0_1_img_c)/(0.41 * lista_u_star_img[1])
    H_corr_img = 1.15 * 1004 * (a_corr + (b_corr * (tsup - 273.15)))/lista_rah_img[1]
    
    err_rel = abs(lista_rah[1] - lista_rah[0])*100/lista_rah[1]
    print('Rodada',rodadas,' a vale: ', a_corr, 'b vale: ', b_corr, 'O Erro relativo é: ', err_rel)
    print ('H é: ', H)
 
    #excluíndo o primeiro ítem de cada lista
    lista_u_star.pop(0)
    lista_rah.pop(0)
    lista_u_star_img.pop(0)
    lista_rah_img.pop(0)
 
print('Fim do processo, a vale: ', a_corr, 'b vale: ', b_corr, 'O Erro relativo é: ', err_rel)
 
#//Calculando o Fluxo de calor latente (LE) e a ET_24h
 
LE = rn - g_corrigido - H_corr_img
LE = np.where(LE < 0.0, 0.0, LE)
fe = LE / (rn - g_corrigido)
 
del_ = 0.409*np.sin(2*np.pi*DOA/(365-1.39))
fi = lat * np.pi/180
H_hor = np.arccos(-np.tan(fi)*np.tan(del_))
rs_24_toa = 118.08*dr*(H_hor*np.sin(fi)*np.sin(del_)+np.cos(fi)*np.cos(del_)*np.sin(H_hor))/np.pi
print('Rs_TOA é: ', rs_24_toa)
 
tsw_24 = rs_24 / rs_24_toa
print('tsw_24 é: ', tsw_24)
 
rn_24 = (1-albedo)*rs_24*11.57407 - 110*tsw_24 #//Rs_24 transformado em W/m²
 
#print('LE é: ', LE)
ET_24 = 0.0353*fe*rn_24
ET_24 = np.ma.masked_where(ET_24 < 0, ET_24)
ET_24 = ET_24.filled(-99)
 
f1_frio = f1_frio.filled(-99)
out_ds = make_raster(albedo_open, 'le', LE, gdal.GDT_Float32, -99)
#out_ds = make_raster(albedo_open, 'le', LE, gdal.GDT_Float32, -99)
#out_ds = make_raster(albedo_open, export_ndvi, ndvi, gdal.GDT_Float32, -99)
out_ds = make_raster(albedo_open, 'g', g_corrigido, gdal.GDT_Float32, -99)
out_ds = make_raster(albedo_open, 'rn', rn, gdal.GDT_Float32, -99)
out_ds = make_raster(albedo_open, 'h', H_corr_img, gdal.GDT_Float32, -99)
out_ds = make_raster(albedo_open, export_et24, ET_24, gdal.GDT_Float32, -99)
 
del out_ds

1.0613773239067819
dr é:  1.03505
Cos(z) é:  0.919720361134887
elevacao é:  66.88523482351987
e_a equivale a :  1.6377583583068727
w equivale a :  22.185603806054523
transmitancia equivale a :  0.7613093073560777
a radiação global é:  990.7105429937882
a radiação atm é:  396.1995749394879
Esse é o percentil 95 do NDVI  0.9246758357562967
esse é o percentil 20 da Tsup:  298.6078002929687
albedo chave é:  0.18313208761022004
Esse é o pixel frio do CIMEC:  297.18829345703125
Esse é o pixel quente do CIMEC:  306.60516357421875
O savi do Pixel quente é:  0.029252122787584982
O Rn do Pixel quente é:  751.9114780422176
O G do Pixel quente é:  225.57344341266528
O Fluxo de calor sensível H é:  526.3380346295521
Rn - G é:  526.3380346295523
a vale:  -7.221938765473871 b vale:  0.3004347533398396 O Erro relativo é:  85.41619996702153
H é:  526.3380346295523
Rodada 2  a vale:  -15.980556746113937 b vale:  0.664795809015285 O Erro relativo é:  54.80796520290157
H é:  526.3380346295525
Rodada 3  a 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:121: RuntimeWarning: invalid value encountered in log


In [ ]:
########################################################       DICAS     #######################################
#Reducer no GEE:
var cos_z_red = cos_z_img.reduceRegion({
  reducer: ee.Reducer.mean(),
  geometry: area,
  scale: 30
});
#Reducer no python
cos_z_red = cos_z_img.reduceRegion(
  ee.Reducer.mean(),
  area,
  30)

#pegando um valor do Dict no GEE:
var cos_z = cos_z_red.getNumber('SolarZenith');

#pegando um valor do Dict no python:
cos_z = cos_z_red.getInfo()['SolarZenith']